## GAN Basics

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [15]:
"""
simple descriminator network with 2 layer NN
"""
class Discriminator(nn.Module):
    def __init__(self,input_dim):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(input_dim,128),
            nn.LeakyReLU(0.01),
            nn.Linear(128,1),
            nn.Sigmoid()
        )
        
    def forward(self,x):
        return self.disc(x)

In [16]:
"""
Generator network
use tanh to normalize inputs to [-1, 1] so make outputs [-1, 1]
"""
class Generator(nn.Module):
    def __init__(self,in_dim,out_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(in_dim,256),
            nn.LeakyReLU(0.01),
            nn.Linear(256,out_dim),
            nn.Tanh()
        )
        
    def forward(self,x):
        return self.gen(x)

In [18]:
# hyperparameter
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64
image_dim = 28 * 28 * 1  # 784
batch_size = 32
num_epochs = 50

disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5,), (0.5,))
    ])

In [19]:
dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"logs/fake")
writer_real = SummaryWriter(f"logs/real")
step = 0

100%|████████████████████████████████████████████████| 9912422/9912422 [00:01<00:00, 8603270.96it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 4224288.39it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|████████████████████████████████████████████████| 1648877/1648877 [00:01<00:00, 1431766.71it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 3339268.85it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [20]:
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        # where the second option of maximizing doesn't suffer from
        # saturating gradients
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1

Epoch [0/50] Batch 0/1875                       Loss D: 0.7007, loss G: 0.7174
Epoch [1/50] Batch 0/1875                       Loss D: 0.6941, loss G: 0.7301
Epoch [2/50] Batch 0/1875                       Loss D: 0.7744, loss G: 0.7698
Epoch [3/50] Batch 0/1875                       Loss D: 0.5293, loss G: 1.3792
Epoch [4/50] Batch 0/1875                       Loss D: 0.8909, loss G: 0.7155
Epoch [5/50] Batch 0/1875                       Loss D: 0.6103, loss G: 1.4277
Epoch [6/50] Batch 0/1875                       Loss D: 0.4111, loss G: 1.3907
Epoch [7/50] Batch 0/1875                       Loss D: 0.6781, loss G: 0.7968
Epoch [8/50] Batch 0/1875                       Loss D: 0.3731, loss G: 1.6038
Epoch [9/50] Batch 0/1875                       Loss D: 0.7416, loss G: 0.9327
Epoch [10/50] Batch 0/1875                       Loss D: 0.4722, loss G: 1.8581
Epoch [11/50] Batch 0/1875                       Loss D: 0.5634, loss G: 1.3538
Epoch [12/50] Batch 0/1875                       L